In [1]:
import sys
sys.path.append("..")
import pickle

In [2]:
from games.abstract_poker import AbstractPokerRootChanceGameState
from games.algorithms import ChanceSamplingCFR, VanillaCFR
from common.constants import card_dealing, results_map
import numpy as np
import pandas as pd

from games.competition_abstract_poker import CompetitionAbstractPoker
from copy import deepcopy

In [3]:
# Hyperparameters
_NUM_TRAIN_ITERS = 100_000
_COMPETITION_ROUNDS = 100_000
_RUN_SINGLE_EXAMPLE = False

# Try single example

In [4]:
if _RUN_SINGLE_EXAMPLE:
    bots = []

    root = AbstractPokerRootChanceGameState(max_hand_strength = 3, 
                                        max_turns = 1,
                                        report = False)
    chance_sampling_cfr = ChanceSamplingCFR(root)
    chance_sampling_cfr.run(iterations = 100)
    chance_sampling_cfr.compute_nash_equilibrium()

    bots.append(chance_sampling_cfr)

    root = AbstractPokerRootChanceGameState(max_hand_strength = 3, 
                                        max_turns = 2,
                                        report = False)
    chance_sampling_cfr = ChanceSamplingCFR(root)
    chance_sampling_cfr.run(iterations = 100)
    chance_sampling_cfr.compute_nash_equilibrium()

    bots.append(chance_sampling_cfr)

    results = []
    rounds_per_competition = 1

    c = CompetitionAbstractPoker(deepcopy(bots[0]), deepcopy(bots[1]), max_turns = 2, report = True)
    result = c.compete(rounds = rounds_per_competition)
    print(f"Result is {result}")

# Granularity competition

In [5]:
def train_bots_fixed_depth(granularity, depth, report = True):
    bots = []

    for i in granularity:
        root = AbstractPokerRootChanceGameState(max_hand_strength = i, 
                                            max_turns = depth,
                                            report = False)
        chance_sampling_cfr = ChanceSamplingCFR(root)
        chance_sampling_cfr.run(iterations = _NUM_TRAIN_ITERS)
        chance_sampling_cfr.compute_nash_equilibrium()

        # vanilla cfr
        # vanilla_cfr_g3 = VanillaCFR(root)
        # vanilla_cfr_g3.run(iterations = _NUM_TRAIN_ITERS)
        # vanilla_cfr_g3.compute_nash_equilibrium()

        bots.append(chance_sampling_cfr)
        
    if report:
        print("------------------------------------------------------------")
        for b in bots:
            print(f"Value of game for granularity {b.root.max_hand_strength}: {b.value_of_the_game():.5f}")
        
    return bots

## Train static bots

In [6]:
static_bots = train_bots_fixed_depth(granularity = [3,5,7,10], depth = 1)
dynamic_bots = train_bots_fixed_depth(granularity = [3,5,7,10], depth = 2)

------------------------------------------------------------
Value of game for granularity 3: -0.02816
Value of game for granularity 5: -0.04350
Value of game for granularity 7: -0.04882
Value of game for granularity 10: -0.05371
------------------------------------------------------------
Value of game for granularity 3: -0.02248
Value of game for granularity 5: -0.03323
Value of game for granularity 7: -0.05133
Value of game for granularity 10: -0.05386


## Competition

In [7]:
def run_competition(bots, game_depth, rounds, report):
    results = []

    for b1 in bots:
        for b2 in bots:
            g1 = b1.root.max_hand_strength
            g2 = b2.root.max_hand_strength
            c = CompetitionAbstractPoker(deepcopy(b1), deepcopy(b2), max_turns = game_depth, report = False)
            result = c.compete(rounds = rounds)
            results.append([g1,g2,result])
            
            if report:
                print(f"{g1} vs {g2}. Result is {result}")
                
    return results

In [8]:
static_competition = run_competition(static_bots, game_depth = 1, rounds = _COMPETITION_ROUNDS, report = True)

3 vs 3. Result is -0.02192
3 vs 5. Result is -0.04238
3 vs 7. Result is -0.04907
3 vs 10. Result is -0.05726
5 vs 3. Result is -0.00804
5 vs 5. Result is -0.04073
5 vs 7. Result is -0.04661
5 vs 10. Result is -0.05783
7 vs 3. Result is -0.0152
7 vs 5. Result is -0.03968
7 vs 7. Result is -0.05683
7 vs 10. Result is -0.04929
10 vs 3. Result is -0.02779
10 vs 5. Result is -0.04585
10 vs 7. Result is -0.04705
10 vs 10. Result is -0.04861


In [9]:
dynamic_competition = run_competition(dynamic_bots, game_depth = 2, rounds = _COMPETITION_ROUNDS, report = True)

3 vs 3. Result is -0.02824
3 vs 5. Result is -0.06019
3 vs 7. Result is -0.05542
3 vs 10. Result is -0.04817
5 vs 3. Result is 0.00715
5 vs 5. Result is -0.02454
5 vs 7. Result is -0.05641
5 vs 10. Result is -0.05563
7 vs 3. Result is -0.01157
7 vs 5. Result is -0.04915
7 vs 7. Result is -0.04504
7 vs 10. Result is -0.05075
10 vs 3. Result is -0.01119
10 vs 5. Result is -0.04492
10 vs 7. Result is -0.05087
10 vs 10. Result is -0.0501


# Export results 

In [10]:
static_df = pd.DataFrame(static_competition, columns = ['player1_granularity', 'player2_granularity', 'result'])\
    .assign(bot_depth = 1,
            game_depth = 1)
dynamic_df = pd.DataFrame(dynamic_competition, columns = ['player1_granularity', 'player2_granularity', 'result'])\
    .assign(bot_depth = 2,
            game_depth = 2)

In [11]:
results_df = pd.concat([static_df, dynamic_df], ignore_index = True)

In [12]:
results_df.head(3)

,player1_granularity,player2_granularity,result,bot_depth,game_depth
0,3,3,-0.02192,1,1
1,3,5,-0.04238,1,1
2,3,7,-0.04907,1,1


In [14]:
results_df.to_csv("output/bot_competition_by_granularity.csv", index = False)